## chapter 10 scratch

In [1]:
import numpy as np
from sklearn.datasets import load_iris 
from sklearn.linear_model import Perceptron

iris = load_iris()
X = iris.data[:, (2, 3)] # petal length, petal width 
y = (iris.target == 0).astype(np.int) # Iris Setosa?

In [2]:
per_clf = Perceptron(random_state=42)
per_clf.fit(X, y)

Perceptron(alpha=0.0001, class_weight=None, eta0=1.0, fit_intercept=True,
      n_iter=5, n_jobs=1, penalty=None, random_state=42, shuffle=True,
      verbose=0, warm_start=False)

In [3]:
y_pred = per_clf.predict([[2, 0.5]])
print(y_pred)

[1]


### creating mnist model in tf

In [4]:
import tensorflow as tf

In [5]:
n_inputs = 28*28 # MNIST 
n_hidden1 = 300 
n_hidden2 = 100 
n_outputs = 10

In [6]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

In [7]:
def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="kernel")
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.matmul(X, W) + b
        if activation is not None:
            return activation(Z)
        else:
            return Z

In [8]:
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    logits = neuron_layer(hidden2, n_outputs, name="outputs")

In [9]:
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")

In [10]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [11]:
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [12]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [13]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

### running mnist

In [14]:
from tensorflow.examples.tutorials.mnist import input_data 
mnist = input_data.read_data_sets("/tmp/data/")

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [15]:
n_epochs = 40
batch_size = 50

with tf.Session() as sess:
    
    init.run()
    
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X:X_batch, y:y_batch})
        acc_train = accuracy.eval(feed_dict={X:X_batch, y:y_batch})
        acc_test = accuracy.eval(feed_dict={X:mnist.test.images, y:mnist.test.labels})
        
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)
        
    save_path = saver.save(sess, "./my_model_final.ckpt")

0 Train accuracy: 0.88 Test accuracy: 0.9016
1 Train accuracy: 0.86 Test accuracy: 0.9206
2 Train accuracy: 0.92 Test accuracy: 0.9308
3 Train accuracy: 0.96 Test accuracy: 0.9373
4 Train accuracy: 0.94 Test accuracy: 0.9418
5 Train accuracy: 0.92 Test accuracy: 0.9446
6 Train accuracy: 0.92 Test accuracy: 0.9495
7 Train accuracy: 0.94 Test accuracy: 0.954
8 Train accuracy: 0.96 Test accuracy: 0.9554
9 Train accuracy: 0.98 Test accuracy: 0.9582
10 Train accuracy: 1.0 Test accuracy: 0.9601
11 Train accuracy: 0.92 Test accuracy: 0.9616
12 Train accuracy: 1.0 Test accuracy: 0.9641
13 Train accuracy: 0.98 Test accuracy: 0.9634
14 Train accuracy: 1.0 Test accuracy: 0.9662
15 Train accuracy: 1.0 Test accuracy: 0.9667
16 Train accuracy: 0.96 Test accuracy: 0.9675
17 Train accuracy: 1.0 Test accuracy: 0.9687
18 Train accuracy: 1.0 Test accuracy: 0.9691
19 Train accuracy: 0.98 Test accuracy: 0.9705
20 Train accuracy: 1.0 Test accuracy: 0.9709
21 Train accuracy: 1.0 Test accuracy: 0.9725
22 Trai